# Food Delivery Hackathon Analysis
Name: Viji  
Date: 31 January 2026


In [7]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt


In [8]:
orders = pd.read_csv('orders.csv')
orders.head()



,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


# Load Users Data (JSON)


In [9]:
users = pd.read_json('users.json')
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


# Load Restaurants Data (SQL)


# Data Inspection & Cleaning


In [16]:
conn = sqlite3.connect(':memory:')
with open('restaurants.sql') as f:
    conn.executescript(f.read())

restaurants = pd.read_sql('SELECT * FROM restaurants', conn)


In [17]:
orders.info()
users.info()
restaurants.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0

In [18]:
orders.isna().sum()
users.isna().sum()
restaurants.isna().sum()


restaurant_id      0
restaurant_name    0
cuisine            0
rating             0
dtype: int64

In [19]:
df = orders.merge(users, on='user_id', how='left') \
           .merge(restaurants, on='restaurant_id', how='left')

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


##  Save Final Dataset


In [20]:
df.to_csv('final_food_delivery_dataset.csv', index=False)


In [23]:
df['order_date'] = pd.to_datetime(df['order_date'],dayfirst=True)
df['month'] = df['order_date'].dt.month
df['quarter'] = df['order_date'].dt.to_period('Q')


In [25]:
# 1. Load the three sources
orders = pd.read_csv('orders.csv') # Based on your image showing an XLS file
users = pd.read_json('users.json')
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

# 2. Merge them together (The "Secret Sauce")
# First join orders with users
intermediate_df = pd.merge(orders, users, on='user_id', how='left')

# Then join with restaurants to create 'final_df'
final_df = pd.merge(intermediate_df, restaurants, on='restaurant_id', how='left')

# Close the database connection
conn.close()

print("final_df is now ready for analysis!")

final_df is now ready for analysis!


In [26]:
# To find the city with the highest revenue from Gold members:
gold_members = final_df[final_df['membership'] == 'Gold']
top_city = gold_members.groupby('city')['total_amount'].sum().idxmax()
print(f"The city with highest Gold member revenue is: {top_city}")

The city with highest Gold member revenue is: Chennai


In [27]:
# Group by cuisine and calculate the mean of total_amount
avg_cuisine_value = final_df.groupby('cuisine')['total_amount'].mean()

# Find the cuisine with the maximum average
print(f"Cuisine with highest average order value: {avg_cuisine_value.idxmax()}")
print(avg_cuisine_value.sort_values(ascending=False))

Cuisine with highest average order value: Mexican
cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64


In [28]:
# Sum the total_amount for each user
user_spending = final_df.groupby('user_id')['total_amount'].sum()

# Count how many users spent more than 1000
high_spenders_count = (user_spending > 1000).sum()
print(f"Number of users who spent > ₹1000: {high_spenders_count}")

Number of users who spent > ₹1000: 2544


In [29]:
# Define the bins and labels for the ratings
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']

# Create a new column for the rating range
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)

# Calculate revenue per range
revenue_by_rating = final_df.groupby('rating_range')['total_amount'].sum()
print(f"Highest revenue rating range: {revenue_by_rating.idxmax()}")
print(revenue_by_rating)

Highest revenue rating range: 4.6 – 5.0
rating_range
3.0 – 3.5    1881754.57
3.6 – 4.0    1717494.41
4.1 – 4.5    1960326.26
4.6 – 5.0    2197030.75
Name: total_amount, dtype: float64


C:\Users\Vijayalakshmi S\AppData\Local\Temp\ipykernel_18080\541313732.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  revenue_by_rating = final_df.groupby('rating_range')['total_amount'].sum()


In [30]:
# Filter for Gold members
gold_members = final_df[final_df['membership'] == 'Gold']

# Group by city and calculate the average total_amount
gold_city_aov = gold_members.groupby('city')['total_amount'].mean()

print(f"City with highest Gold AOV: {gold_city_aov.idxmax()}")
print(gold_city_aov)

City with highest Gold AOV: Chennai
city
Bangalore    793.223756
Chennai      808.459080
Hyderabad    806.421034
Pune         781.162243
Name: total_amount, dtype: float64


In [31]:
# Calculate both count of restaurants and sum of revenue per cuisine
cuisine_stats = final_df.groupby('cuisine').agg(
    restaurant_count=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
)

print(cuisine_stats.sort_values(by='restaurant_count'))

         restaurant_count  total_revenue
cuisine                                 
Chinese               120     1930504.65
Indian                126     1971412.58
Italian               126     2024203.80
Mexican               128     2085503.09


In [32]:
# Group by cuisine to count unique restaurants and sum the revenue
cuisine_analysis = final_df.groupby('cuisine').agg(
    distinct_restaurants=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
)

# Sort by restaurant count to see which is lowest
print(cuisine_analysis.sort_values(by='distinct_restaurants'))

         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Indian                    126     1971412.58
Italian                   126     2024203.80
Mexican                   128     2085503.09


In [33]:
# Q7: Percentage of orders by Gold members
gold_percentage = (len(final_df[final_df['membership'] == 'Gold']) / len(final_df)) * 100
print(f"Percentage of Gold member orders: {round(gold_percentage)}%")

# Q9: Highest revenue combination
combo_revenue = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum()
print(f"Highest revenue combination: {combo_revenue.idxmax()}")

Percentage of Gold member orders: 50%
Highest revenue combination: ('Regular', 'Mexican')


In [34]:
# 1. Calculate the total count of orders and the average order value for each restaurant
restaurant_stats = final_df.groupby('name').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
)

# 2. Filter for restaurants with less than 20 orders
low_volume_restaurants = restaurant_stats[restaurant_stats['total_orders'] < 20]

# 3. Find the one with the highest average order value
winner = low_volume_restaurants['avg_order_value'].idxmax()
winner_value = low_volume_restaurants['avg_order_value'].max()

print(f"The restaurant is: {winner}")
print(f"Average Order Value: ₹{round(winner_value, 2)}")

The restaurant is: User_2429
Average Order Value: ₹1497.42


In [35]:
# Group by both membership and cuisine to calculate total revenue
combo_analysis = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum()

# Sort the results to find the highest
print(combo_analysis.sort_values(ascending=False))

# To find the exact answer automatically:
highest_combo = combo_analysis.idxmax()
print(f"\nThe highest revenue combination is: {highest_combo}")

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

The highest revenue combination is: ('Regular', 'Mexican')


In [36]:
# 1. Ensure order_date is in datetime format (if not done already)
final_df['order_date'] = pd.to_datetime(final_df['order_date'])

# 2. Extract the Quarter from the date
final_df['quarter'] = final_df['order_date'].dt.to_period('Q')

# 3. Sum the revenue for each quarter
quarterly_revenue = final_df.groupby('quarter')['total_amount'].sum()

# 4. Identify the highest quarter
print("Total Revenue per Quarter:")
print(quarterly_revenue)
print(f"\nThe Quarter with the highest revenue is: {quarterly_revenue.idxmax()}")

Total Revenue per Quarter:
quarter
2023Q1    1993425.14
2023Q2    1945348.72
2023Q3    2037385.10
2023Q4    2018263.66
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

The Quarter with the highest revenue is: 2023Q3


C:\Users\Vijayalakshmi S\AppData\Local\Temp\ipykernel_18080\231767217.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


In [37]:
total_gold_orders = len(final_df[final_df['membership'] == 'Gold'])
print(f"Total orders by Gold members: {total_gold_orders}")

Total orders by Gold members: 4987


In [38]:
hyderabad_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"Total revenue in Hyderabad: {round(hyderabad_revenue)}")

Total revenue in Hyderabad: 1889367


In [39]:
distinct_users = final_df['user_id'].nunique()
print(f"Number of distinct users: {distinct_users}")

Number of distinct users: 2883


In [40]:
gold_avg_value = final_df[final_df['membership'] == 'Gold']['total_amount'].mean()
print(f"Average order value for Gold members: {round(gold_avg_value, 2)}")

Average order value for Gold members: 797.15


In [41]:
high_rated_orders = len(final_df[final_df['rating'] >= 4.5])
print(f"Orders for restaurants with rating ≥ 4.5: {high_rated_orders}")

Orders for restaurants with rating ≥ 4.5: 3374


In [42]:
# 1. Filter for Gold members
gold_df = final_df[final_df['membership'] == 'Gold']

# 2. Find the city with the highest total_amount among Gold members
top_gold_city = gold_df.groupby('city')['total_amount'].sum().idxmax()

# 3. Count orders in that specific city for Gold members
orders_in_top_city = len(gold_df[gold_df['city'] == top_gold_city])

print(f"Top Revenue City for Gold: {top_gold_city}")
print(f"Orders in {top_gold_city} by Gold members: {orders_in_top_city}")

Top Revenue City for Gold: Chennai
Orders in Chennai by Gold members: 1337
